In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv('AAPL.csv')
data = df.filter(['Open', 'Close'])
data

KeyboardInterrupt: 

In [ ]:
# scale the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

# split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

test_data

In [ ]:
# create input/output arrays for training and testing data
def create_dataset(data, lookback):
    X, Y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, :])
        Y.append(data[i, 0:2]) # use both "Open" and "Close" values
    return np.array(X), np.array(Y)

lookback = 60
x_train, y_train = create_dataset(train_data, lookback)
x_test, y_test = create_dataset(test_data, lookback)

x_test

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 2)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(2))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error:', mae)

In [ ]:
last_60_days = data[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_price = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price)
print('Next day\'s predicted stock price:', predicted_price)

In [ ]:

import matplotlib.pyplot as plt

plt.plot(y_test, label=True)
plt.plot(predictions, label="LSTM Value")
plt.plot(predicted_price, label="Predicted Price Value")
plt.title("Prediction by LSTM")
plt.xlabel("Time Scale")
plt.ylabel("Scaled USD")
plt.legend()
plt.show()